In [1]:
import pandas as pd
import datetime
import datetime
import os
from haversine import haversine
import glob
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

IRI_Selected_18=['LOS ANGELES','DALLAS/FT WORTH','NEW YORK','HOUSTON','WEST TEX/NEW MEXICO','PHOENIX/TUCSON','SOUTH CAROLINA',
                'BIRMINGHAM/MONTGOMERY','MISSISSIPPI','BALTIMORE/WASHINGTON','NEW ORLEANS/MOBILE',
                 'CHICAGO','ATLANTA','DETROIT','MIAMI/FT LAUDERDALE','HARRISBURG/SCRANTON','BOSTON',"ROANOKE"] #'RALEIGH/GREENSBORO','DENVER'
len(IRI_Selected_18)

18

# IRI zips for the 18 Mkts

In [2]:
IRI_Counties_both=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Sheet1',dtype=str)
IRI_Counties_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='IRI',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Nielsen',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen.columns=[['County','cleaned_name']]
IRI_Counties_both=IRI_Counties_both[(IRI_Counties_both['IRI_Label']!='nan') & (IRI_Counties_both['Nielsen_Label']!='nan')]

IRI_Counties_both=IRI_Counties_both[['County','County']]
IRI_Counties_both.columns=['County','cleaned_name']

Nielsen_counties=IRI_Counties_both.append(IRI_Counties_Nielsen).drop_duplicates().reset_index()
IRI_counties=IRI_Counties_both.append(IRI_Counties_IRI).drop_duplicates().reset_index()

del Nielsen_counties['index']
del IRI_counties['index']


In [3]:
All_DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
All_DMA_zips=All_DMA_zips.iloc[:,[0,2,6,9]]
All_DMA_zips.columns=['zip_code','DMA','State','County']
All_DMA_zips['County']=All_DMA_zips['County']+"|"+All_DMA_zips['State']
All_DMA_zips['County']=All_DMA_zips['County'].apply(lambda x: x.replace(" ","").lower())
All_DMA_zips=All_DMA_zips[~All_DMA_zips['State'].isin(['AK','HI'])]

In [4]:
All_IRI_counties=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",dtype=str,sheetname='U.S. Counties',skiprows=1)
All_IRI_counties=All_IRI_counties[['County Name','State','IRI Market']]
All_IRI_counties['County']=All_IRI_counties['County Name']+"|"+All_IRI_counties['State']
All_IRI_counties['County']=All_IRI_counties['County'].apply(lambda x: x.replace(" ","").lower())
All_IRI_counties=pd.merge(All_IRI_counties,IRI_counties,on="County",how="left")
All_IRI_counties=All_IRI_counties[['IRI Market','cleaned_name']]

In [5]:
All_DMA_zips=pd.merge(All_DMA_zips,Nielsen_counties,on="County",how="left")
All_DMA_zips=All_DMA_zips[['zip_code','DMA','County','cleaned_name']].rename(columns={"County":"Nielsen_County"})
All_DMA_zips_merge=pd.merge(All_DMA_zips,All_IRI_counties,on="cleaned_name",how="left")


In [6]:
IRI_Zip_with_dup=All_DMA_zips_merge[['IRI Market','zip_code']].drop_duplicates()

In [7]:
IRI_Zip_18=IRI_Zip_with_dup[IRI_Zip_with_dup['IRI Market'].isin(IRI_Selected_18)]

IRI_Zip_others=IRI_Zip_with_dup[~IRI_Zip_with_dup['IRI Market'].isin(IRI_Selected_18)]
IRI_Zip_others=IRI_Zip_others[~IRI_Zip_others['zip_code'].isin(IRI_Zip_18['zip_code'].tolist())]
IRI_Zip_others=IRI_Zip_others[IRI_Zip_others['IRI Market']!="NO MARKET"]

IRI_Zip_NoMkt=IRI_Zip_with_dup[IRI_Zip_with_dup['IRI Market']=="NO MARKET"]
IRI_Zip_NoMkt=IRI_Zip_NoMkt[~IRI_Zip_NoMkt['zip_code'].isin(IRI_Zip_18['zip_code'].tolist())]
IRI_Zip_NoMkt=IRI_Zip_NoMkt[~IRI_Zip_NoMkt['zip_code'].isin(IRI_Zip_others['zip_code'].tolist())]

print(IRI_Zip_18.shape)
print(IRI_Zip_18['IRI Market'].nunique())
print(IRI_Zip_18['zip_code'].nunique())


print(IRI_Zip_others.shape)
print(IRI_Zip_others['IRI Market'].nunique())
print(IRI_Zip_others['zip_code'].nunique())

IRI_zip_18_and_the_rest=IRI_Zip_18.append(IRI_Zip_others).append(IRI_Zip_NoMkt)
print(IRI_zip_18_and_the_rest.shape)
# IRI_zip_18_and_the_rest.groupby("IRI Market")['zip_code'].count().to_frame().reset_index().sort_values("zip_code",ascending=False)

(10157, 2)
18
10149
(12009, 2)
46
11961
(40610, 2)


# Walmarts in the zips above from SafeGraph

In [8]:
latest_SG_brand_info=pd.read_csv("/home/jian/Docs/SafeGraph/places_20190629/brand_info/part-00000-tid-3844431843075532209-d890661b-f976-44e8-9678-725e8cd08c6f-119-c000.csv.gz",
                                dtype=str,compression="gzip",sep="|")
[x for x in latest_SG_brand_info['brand_name'].unique().tolist() if "almart" in x.lower()]

['Walmart', 'Walmart inc']

In [9]:
POI_data=glob.glob("/home/jian/Docs/SafeGraph/places_20190629/poi/*csv.gz")
POI_data

['/home/jian/Docs/SafeGraph/places_20190629/poi/part-00001-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-112-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00003-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-114-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00000-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-111-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00004-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-115-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00002-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-113-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00005-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-116-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places_20190629/poi/part-00007-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-118-c000.csv.gz',
 '/home/jian/Docs/SafeGraph/places

In [10]:
all_SG_WMT_POI=pd.DataFrame()
all_SG_WMTinc_POI=pd.DataFrame()

for file in POI_data:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep="|")
    df_Walmart=df[df['brands']=="Walmart"]
    df_Walmartinc=df[df['brands']=="Walmart inc"]
    
    all_SG_WMT_POI=all_SG_WMT_POI.append(df_Walmart)
    all_SG_WMTinc_POI=all_SG_WMTinc_POI.append(df_Walmartinc)
    
    print(datetime.datetime.now(),file)

2019-09-25 17:20:43.787462 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00001-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-112-c000.csv.gz
2019-09-25 17:21:41.998873 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00003-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-114-c000.csv.gz
2019-09-25 17:21:54.271612 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00000-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-111-c000.csv.gz
2019-09-25 17:22:04.501787 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00004-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-115-c000.csv.gz
2019-09-25 17:22:15.047481 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00002-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-113-c000.csv.gz
2019-09-25 17:22:25.149227 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00005-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-116-c000.csv.gz
2019-09-25 17:22:34.863639 /home/j

In [11]:
print(all_SG_WMT_POI.shape)
print(all_SG_WMTinc_POI.shape)

# all_SG_WMT_POI

(4669, 20)
(0, 20)


In [12]:
all_SG_WMT_POI['brands'].unique()

array(['Walmart'], dtype=object)

In [13]:
df_slicing_Wmts=all_SG_WMT_POI.groupby("location_name")['safegraph_place_id'].count().to_frame().reset_index().sort_values("safegraph_place_id")
included_location_names_list=df_slicing_Wmts[df_slicing_Wmts['safegraph_place_id']>=10]['location_name'].tolist()
print(included_location_names_list)

all_SG_WMT_POI=all_SG_WMT_POI[all_SG_WMT_POI["location_name"].isin(included_location_names_list)]
all_SG_WMT_POI['zip_code']=all_SG_WMT_POI['zip_code'].apply(lambda x: x.zfill(5))
all_SG_WMT_POI.shape

['Walmart Supercenter', 'Wal Mart Supercenter', 'Walmart']


(4653, 20)

In [14]:
'''
df_zip_by_18_mkt_count=IRI_Zip_18.groupby("zip_code")['IRI Market'].nunique().to_frame().reset_index().rename(columns={"IRI Market":"count_mkts"}).sort_values("count_mkts",ascending=False)
df_zip_by_18_mkt_list=IRI_Zip_18.groupby("zip_code")['IRI Market'].apply(list).to_frame().reset_index()

df_zip_by_18_mkts=pd.merge(df_zip_by_18_mkt_count,df_zip_by_18_mkt_list,on="zip_code",how="outer")
df_zip_by_18_mkts.shape
'''

df_zip_by_18_and_the_rest=IRI_zip_18_and_the_rest.groupby("zip_code")['IRI Market'].apply(list).to_frame().reset_index().rename(columns={"IRI Market":"IRI_list"})
df_zip_by_18_and_the_rest_2=IRI_zip_18_and_the_rest.groupby("zip_code")['IRI Market'].nunique().to_frame().reset_index().rename(columns={"IRI Market":"count_mkts"}).sort_values("count_mkts",ascending=False)
df_zip_by_18_and_the_rest=pd.merge(df_zip_by_18_and_the_rest,df_zip_by_18_and_the_rest_2,on="zip_code",how="outer")
df_zip_by_18_and_the_rest['IRI_list']=df_zip_by_18_and_the_rest['IRI_list'].apply(lambda x: list(set(x)))
del df_zip_by_18_and_the_rest_2
df_zip_by_18_and_the_rest.shape

(40554, 3)

In [15]:
walmarts_with_IRIs=pd.merge(all_SG_WMT_POI,df_zip_by_18_and_the_rest,on="zip_code",how="left")
walmarts_with_IRIs['IRI_list']=walmarts_with_IRIs['IRI_list'].fillna("['miss_matched']")
walmarts_with_IRIs['IRI_list']=walmarts_with_IRIs['IRI_list'].astype(str)
walmarts_with_IRIs['IRI_list']=walmarts_with_IRIs['IRI_list'].apply(eval)
walmarts_with_IRIs['latitude']=walmarts_with_IRIs['latitude'].astype(float)
walmarts_with_IRIs['longitude']=walmarts_with_IRIs['longitude'].astype(float)

walmarts_with_IRIs.head(2)

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,...,state,zip_code,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,IRI_list,count_mkts
0,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,ca,95776,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,[SACRAMENTO],1.0
1,sg:a34b1824bf7c4d758634a35173905c85,NaN,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,28.055297,-80.623241,...,fl,32901,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...","POLYGON ((-80.6234859 28.055599800000003, -80....",OWNED_POLYGON,NaN,false,NaN,[ORLANDO],1.0


In [16]:
# 1 store in IL 60418 is missing matching and PR, AK, HI are also missing matching

In [17]:
walmarts_with_IRIs_iter=walmarts_with_IRIs[['safegraph_place_id','latitude','longitude']]

In [18]:
df_mapping_store_10_zips=pd.DataFrame()
for ind,row in walmarts_with_IRIs_iter.iterrows():
    store_coor=[row['latitude'],row['longitude']]
    store_id=row['safegraph_place_id']
    for zip_cd in zip_centers.keys():
        dist=haversine(store_coor,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"zip_cd":zip_cd,"dist_miles":dist},index=[store_id])
            df_mapping_store_10_zips=df_mapping_store_10_zips.append(df)

In [19]:
df_mapping_store_10_zips=df_mapping_store_10_zips.reset_index().rename(columns={"index":"safegraph_place_id"})

In [20]:
df_output=pd.merge(walmarts_with_IRIs,df_mapping_store_10_zips,on="safegraph_place_id",how="left")
df_output=df_output.rename(columns={"zip_code":"store_zip"})

df_output.head(2)

,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,...,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,IRI_list,count_mkts,dist_miles,zip_cd
0,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,[SACRAMENTO],1.0,7.889054,95618
1,sg:3d14eae49ae24306a6db19a386c658b0,sg:5c052cee6ebf4ba0acc781a7218470ab,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,452319,38.660866,-121.763446,...,"{ ""Mon"": [[""6:00"", ""24:00""]], ""Tue"": [[""6:00"",...",POLYGON ((-121.76448965459713 38.6618353425036...,OWNED_POLYGON,NaN,false,NaN,[SACRAMENTO],1.0,1.005072,95776


In [21]:
df_output=df_output[['IRI_list','safegraph_place_id','zip_cd','dist_miles']+[x for x in df_output.columns.tolist() if x not in ['IRI_list','safegraph_place_id','zip_cd','dist_miles'] ]]
df_output['iri_list_str']=df_output['IRI_list'].astype(str)
df_output=df_output.sort_values(['iri_list_str','safegraph_place_id','dist_miles'])
df_output.shape

(96993, 25)

In [23]:
# Get the market centroid and allocate to the nearest mkt if in multiple mkts
df_mkt_store=df_output[['safegraph_place_id','latitude','longitude','iri_list_str']].drop_duplicates()
df_mkt_store['iri_list']=df_mkt_store['iri_list_str'].apply(eval)
df_mkt_store=df_mkt_store.reset_index()
del df_mkt_store['index']
mkt_centroid=pd.DataFrame()
for ind, row in df_mkt_store.iterrows():
    mkt_list=row['iri_list']
    safegraph_place_id=row['safegraph_place_id']
    latitude=row['latitude']
    longitude=row['longitude']
    
    df=pd.DataFrame({"mkt":mkt_list,
                     "safegraph_place_id":[safegraph_place_id]*len(mkt_list),
                     "latitude":[latitude]*len(mkt_list),
                     "longitude":[longitude]*len(mkt_list)
                    },index=[0]*len(mkt_list))
    mkt_centroid=mkt_centroid.append(df)
mkt_centroid=mkt_centroid.reset_index()
del mkt_centroid['index']
print(mkt_centroid.shape)

mkt_centroid=mkt_centroid.groupby(['mkt'])['latitude','longitude'].mean().reset_index()

mkt_centroid['centroid']=mkt_centroid[['latitude','longitude']].values.tolist()
mkt_centroid_dict=mkt_centroid[(mkt_centroid['mkt']!="NO MARKET") & (mkt_centroid['mkt']!="miss_matched")].set_index("mkt").to_dict()['centroid']


(4661, 4)


In [24]:
allocated_mkt_for_store=df_output[['safegraph_place_id','iri_list_str','latitude','longitude']].drop_duplicates()
print(allocated_mkt_for_store.shape)
allocated_mkt_for_store.head(2)

(4653, 4)


,safegraph_place_id,iri_list_str,latitude,longitude
31622,sg:13d741a918de4e1aae92fa5d275f3c39,['ALBANY'],42.785604,-73.941729
58755,sg:154487f4a28d4de6a408efcd3bfca115,['ALBANY'],42.641576,-73.701794


In [25]:
def select_main_mkt(list_x_str,store_lat,store_long):
    list_x=eval(list_x_str)
    if len(list_x)==1:
        return list_x[0]
    elif len(list_x)==0:
        return nan
    else:
        intersect_with_18=set(list_x).intersection(IRI_Selected_18)
        if len(intersect_with_18)==1:
            return intersect_with_18[0]
        elif len(intersect_with_18)>1:
            select_mkt=None
            min_dist=99999999999
            for mkt_v in intersect_with_18:
                dist=haversine(mkt_centroid_dict[mkt_v],(store_lat,store_long),miles=True)
                if dist<min_dist:
                    min_dist=dist
                    select_mkt=mkt_v
            return select_mkt
        else: # no overlap with 18
            try:
                list_x.remove("NO MARKET")
            except:
                pass
            try:
                list_x.remove("miss_matched")
            except:
                pass
            
            select_mkt=None
            min_dist=99999999999
            for mkt_v in list_x:
                dist=haversine(mkt_centroid_dict[mkt_v],(store_lat,store_long),miles=True)
                if dist<min_dist:
                    min_dist=dist
                    select_mkt=mkt_v
            return select_mkt


In [26]:
allocated_mkt_for_store['allocated_mkt']=allocated_mkt_for_store.apply(lambda x: select_main_mkt(x['iri_list_str'],x['latitude'],x['longitude']),axis=1)


In [27]:
allocated_mkt_for_store.head(2)

,safegraph_place_id,iri_list_str,latitude,longitude,allocated_mkt
31622,sg:13d741a918de4e1aae92fa5d275f3c39,['ALBANY'],42.785604,-73.941729,ALBANY
58755,sg:154487f4a28d4de6a408efcd3bfca115,['ALBANY'],42.641576,-73.701794,ALBANY


In [28]:
allocated_mkt_for_store['iri_list_str'].tolist()[0]

"['ALBANY']"

In [29]:
allocated_multiple_mkt_stores=allocated_mkt_for_store[allocated_mkt_for_store['iri_list_str'].apply(lambda x: len(eval(x))>1)]
all_allocated_mkt_stores=allocated_mkt_for_store[['safegraph_place_id','allocated_mkt']].drop_duplicates()
all_allocated_mkt_stores.shape

(4653, 2)

In [30]:
df_output=pd.merge(df_output,all_allocated_mkt_stores,on="safegraph_place_id",how="left")
df_output.shape

(96993, 26)

In [31]:
df_output.head(2)

,IRI_list,safegraph_place_id,zip_cd,dist_miles,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,...,store_zip,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,count_mkts,iri_list_str,allocated_mkt
0,[ALBANY],sg:13d741a918de4e1aae92fa5d275f3c39,12303,1.16109,sg:6b5d2b26eabc4061b1da3f4b72f37770,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,...,12303,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-73.9423194 42.785352800000005, -73....",OWNED_POLYGON,NaN,false,NaN,1.0,['ALBANY'],ALBANY
1,[ALBANY],sg:13d741a918de4e1aae92fa5d275f3c39,12307,1.43240,sg:6b5d2b26eabc4061b1da3f4b72f37770,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,...,12303,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-73.9423194 42.785352800000005, -73....",OWNED_POLYGON,NaN,false,NaN,1.0,['ALBANY'],ALBANY


In [32]:
# Use the allocated only 1 market
import numpy as np
df_output_unique_zip_by_IRI=df_output[['allocated_mkt','zip_cd']].drop_duplicates()
df_output_unique_zip_by_IRI.shape,df_output_unique_zip_by_IRI['zip_cd'].nunique()
df_output_unique_zip_by_IRI['market_type']=np.where(df_output_unique_zip_by_IRI['allocated_mkt'].isin(IRI_Selected_18),"selected_18",
                                                   np.where(df_output_unique_zip_by_IRI['allocated_mkt'].isin(["NO MARKET",'miss_matched']),df_output_unique_zip_by_IRI['allocated_mkt'],
                                                            "others_64mkts_not_selected"
                                                           )
                                                   )

In [33]:
df_output.head(2)

,IRI_list,safegraph_place_id,zip_cd,dist_miles,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,...,store_zip,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,count_mkts,iri_list_str,allocated_mkt
0,[ALBANY],sg:13d741a918de4e1aae92fa5d275f3c39,12303,1.16109,sg:6b5d2b26eabc4061b1da3f4b72f37770,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,...,12303,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-73.9423194 42.785352800000005, -73....",OWNED_POLYGON,NaN,false,NaN,1.0,['ALBANY'],ALBANY
1,[ALBANY],sg:13d741a918de4e1aae92fa5d275f3c39,12307,1.43240,sg:6b5d2b26eabc4061b1da3f4b72f37770,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,Walmart,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores,...,12303,"{ ""Mon"": [[""7:00"", ""19:00""]], ""Tue"": [[""7:00"",...","POLYGON ((-73.9423194 42.785352800000005, -73....",OWNED_POLYGON,NaN,false,NaN,1.0,['ALBANY'],ALBANY


In [34]:
store_count_by_mkt=df_output.groupby("allocated_mkt")['safegraph_place_id'].nunique().to_frame().reset_index()
store_count_by_mkt['market_type']=np.where(store_count_by_mkt['allocated_mkt'].isin(IRI_Selected_18),"selected_18",
                                                   np.where(store_count_by_mkt['allocated_mkt'].isin(["NO MARKET",'miss_matched']),store_count_by_mkt['allocated_mkt'],
                                                            "others_64mkts_not_selected"
                                                           )
                                                   )
store_count_by_mkt=store_count_by_mkt.sort_values(["market_type",'allocated_mkt'],ascending=[False,True])
# store_count_by_mkt

# Add demo

In [35]:
excel_demo=pd.ExcelFile("./SunnyD_target_demo_by_zip_JL_2019-09-05.xlsx")
excel_demo.sheet_names

['based_on_EASI_zip4', 'based_on_census', 'both_sidebyside']

In [36]:
# Census
df_census_demo=excel_demo.parse("based_on_census")
df_census_demo['ZIP_CODE']=df_census_demo['ZIP_CODE'].apply(lambda x: str(x).zfill(5))
df_census_demo=df_census_demo[['ZIP_CODE','teenagers_10_19_census','calculated_Monther_census','HH_inc_avg']]
df_census_demo=df_census_demo.rename(columns={"ZIP_CODE":"zip_cd"})

In [37]:
df_output_unique_zip_by_IRI_agg_census=pd.merge(df_output_unique_zip_by_IRI,df_census_demo,on="zip_cd",how="left")

In [38]:
unique_zips=df_output_unique_zip_by_IRI_agg_census[['zip_cd']].drop_duplicates()

In [39]:
writer=pd.ExcelWriter("./df_zips_10_miles_around_all_walmarts_labled_IRI_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
# unique_zips.to_excel(writer,"unique_zips",index=False) # only unique 
df_output_unique_zip_by_IRI_agg_census.to_excel(writer,"zips_by_market_with_dup",index=False) # only unique as store in maket
df_output.to_excel(writer,"all_zips_with_store_info_POI",index=False)
store_count_by_mkt.to_excel(writer,"store_count_by_mkt",index=False)
all_allocated_mkt_stores.to_excel(writer,"allocated_mkt_by_store",index=False)
allocated_multiple_mkt_stores.to_excel(writer,"multiple_mkt_case",index=False)
writer.save()

In [40]:
df_output_unique_zip_by_IRI.shape

(24708, 3)

In [41]:
df_output_unique_zip_by_IRI['zip_cd'].nunique()

23882

In [42]:
df_output_unique_zip_by_IRI.head(2)

,allocated_mkt,zip_cd,market_type
0,ALBANY,12303,others_64mkts_not_selected
1,ALBANY,12307,others_64mkts_not_selected
